# Data Augmentation

### Import

In [2]:
import albumentations as alb
import cv2 
import os
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

### Configurações e constantes

In [6]:
INPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\negative\\nocow_dataset"
ANNOTATION_PATH = os.path.join(INPUT_PATH, "w_nocow_annotations.csv")
OUTPUT_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\negative\\augmantation"
AUG_ANNOTATION_CSV_PATH = os.path.join(OUTPUT_PATH, "final_aug_annotation.csv")


N = 10

### Definição de Função

In [7]:
def loadAnnotations(ANNOTATION_PATH):
    df = pd.read_csv(ANNOTATION_PATH)
    return df

#Cria uma Lista com o Nome de todas as imagens no path de entrada 
def createListNameImageFiles(input_path):

    from pathlib import Path
    images = Path(input_path).glob("*.jpg")
    image_names_list = [str(p) for p in images]

    return image_names_list

# Esta função retonar apenas o nome da imagem a partir de um path
def createNameImage(paths_image):
    name_format = paths_image.split("\\")[-1]
    image_name=name_format.split(".")[0] 
    
    return image_name

# Salva a imagens aumentada
def exportAugmentedImage(augmented_img, OUTPUT_PATH, augmented_img_name):
    OUTPUT_PATH = os.path.join(OUTPUT_PATH, augmented_img_name + ".JPG")
    cv2.imwrite(OUTPUT_PATH, cv2.cvtColor(augmented_img, cv2.COLOR_RGB2BGR))
    
# cria uma lista com os bboxes e a class
def createBBoxesList(image_annotation_df):
    
    qt_object = len(image_annotation_df)
    
    bboxesDf = image_annotation_df[['xmin', 'ymin', 'xmax', 'ymax', 'class']]
    
    bboxes_class = []
    
    bboxes = []
    
    for i in range(qt_object):
        
        value_class = [bboxesDf.iloc[i][0],
                 bboxesDf.iloc[i][1], 
                 bboxesDf.iloc[i][2], 
                 bboxesDf.iloc[i][3], 
                 bboxesDf.iloc[i][4]]
        
        value = [bboxesDf.iloc[i][0],
                 bboxesDf.iloc[i][1], 
                 bboxesDf.iloc[i][2], 
                 bboxesDf.iloc[i][3]]
        
        bboxes_class.append(value_class)
        bboxes.append(value)
         
    return bboxes_class, bboxes


def createAugAnnotationDf(filename, width, height, augmented_bboxes):
    
    qt_object = len(augmented_bboxes)
    
    row_list = []
    
    for i in range(qt_object):
        value = [filename,
                 width,
                 height,
                 augmented_bboxes[i][4],
                 augmented_bboxes[i][0].astype(int),
                 augmented_bboxes[i][1].astype(int),
                 augmented_bboxes[i][2].astype(int),
                 augmented_bboxes[i][3].astype(int),
                 0,
                 0]
        
        row_list.append(value)
        
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
    AugAnnotationDf = pd.DataFrame(row_list, columns=column_name)
    
    return AugAnnotationDf

def generateAugmentation(img, bboxes, augmentor, n, OUTPUT_PATH, image_name):
    
    row_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
    FullAugAnnotationDf = pd.DataFrame(row_list, columns=column_name)
    
    for x in range(n):
        
        
        augmented = augmentor(image=img, bboxes=bboxes)
        
        augmented_image = augmented['image']
        augmented_bboxes = augmented['bboxes']
     
        
        """""augmented_image_bbox = augmented_image
        for obj in augmented_bboxes:
            cv2.rectangle(augmented_image_bbox, (int(obj[0]), int(obj[1])), (int(obj[2]), int(obj[3])), (0, 255, 0), 10)"""
            
        width = augmented_image.shape[1]
        height = augmented_image.shape[0]
        
        augmented_img_name = image_name + "_aug_" + str(x)
        
        exportAugmentedImage(augmented_image, OUTPUT_PATH, augmented_img_name)
        
        AugAnnotationDf = createAugAnnotationDf(augmented_img_name, width, height, augmented_bboxes)
        
        FullAugAnnotationDf = pd.concat([FullAugAnnotationDf, AugAnnotationDf])
    
    #showImage(augmented_image_bbox)
        
    
    return FullAugAnnotationDf

# Esta função recebe uma imagem e realiza a vizualização
def showImage(imagem):
    print("Visualizando a imagem...")
    plt.imshow(cv2.cvtColor(imagem, cv2.COLOR_BGR2RGB))
    plt.show()

## Augmentation

In [8]:
def main():
    print("Start Augmentation...")
    
    #create final_full_aug_annotation_df
    row_list = []
    column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
    final_full_aug_annotation_df = pd.DataFrame(row_list, columns=column_name)
    
    # load annotations df
    full_annotations_df = loadAnnotations(ANNOTATION_PATH)
    
    # create image names list 
    image_name_list = createListNameImageFiles(INPUT_PATH)
    
    # Calculate number of images
    qt_images = len(image_name_list)
    
    # define augmentor
    augmentor = alb.Compose([alb.Flip(p=0.7),
                             alb.ShiftScaleRotate(p=0.5),
                             alb.RandomBrightnessContrast(p=0.2),
                             alb.RandomGamma(p=0.2),
                             alb.RGBShift(p=0.1)],
                        bbox_params=alb.BboxParams(format='pascal_voc'))
    
    for image_path in image_name_list:
        
        # load image
        img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)
        
        # coleta o nome da imagem
        image_name = createNameImage(image_path)
        
        #filtra apenas as annotations referentes a imagem
        image_annotation_df = full_annotations_df[full_annotations_df['filename']==image_name]
        
        # cria bboxes
        bboxes_class, bboxes = createBBoxesList(image_annotation_df)
        
    
        
        # realiza o augmentation
        AugAnnotationDf = generateAugmentation(img, bboxes_class, augmentor, N, OUTPUT_PATH, image_name)
        
        final_full_aug_annotation_df = pd.concat([final_full_aug_annotation_df, AugAnnotationDf])
        
    print('Saving final_full_aug_annotation_df')
    
    # Salva o csv com as Annotations de todas as imagens augmentadas
    final_full_aug_annotation_df = final_full_aug_annotation_df.reset_index(drop=True)
    final_full_aug_annotation_df.to_csv(AUG_ANNOTATION_CSV_PATH, encoding='utf-8', index=False)
    
    print('Processo concluido com sucesso..')
    
    
main()

Start Augmentation...
Saving final_full_aug_annotation_df
Processo concluido com sucesso..


### Criando as annotations para as imagens com vacas brancas ou pretas

In [ ]:
BRANCA_IMAGE_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\positive\\augmentation\\pretas"
BRANCA_ANNOTATION_PATH = os.path.join(BRANCA_IMAGE_PATH, "pretas_annotation.csv")

In [ ]:
ori_annotation_df = loadAnnotations(ANNOTATION_PATH)

# create image names list 
image_name_list = createListNameImageFiles(BRANCA_IMAGE_PATH)


#create fina_annotation_df
row_list = []
column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
new_annotation_df = pd.DataFrame(row_list, columns=column_name)


for image_path in image_name_list:

    # coleta o nome da imagem
    image_name = createNameImage(image_path)

    #filtra apenas as annotations referentes a imagem
    image_annotation_df = ori_annotation_df[ori_annotation_df['filename']==image_name]

    new_annotation_df = pd.concat([new_annotation_df, image_annotation_df])

# Salva o csv com as Annotations de todas as imagens augmentadas
new_annotation_df = new_annotation_df.reset_index(drop=True)
new_annotation_df.to_csv(BRANCA_ANNOTATION_PATH, encoding='utf-8', index=False)

print('Processo concluido com sucesso..')  


# augmentation para uma imagem

In [ ]:
DIR_DATASET_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\positive\\augmentation\\brancas"
IMAGE_PATH = os.path.join(DIR_DATASET_PATH, "cow664_w_28.JPG")
ANNOTATION_PATH = os.path.join(DIR_DATASET_PATH, "branca_annotation.csv")
OUTPUT_ANNOTATION_PATH = os.path.join(DIR_DATASET_PATH, "output", "aug_annotation.csv")
OUTPUT_PATH = os.path.join(DIR_DATASET_PATH, "output")

N = 10

In [ ]:
ori_annotation_df = loadAnnotations(ANNOTATION_PATH)
image_path = IMAGE_PATH

#create fina_annotation_df
row_list = []
column_name = ['filename', 'width', 'height','class', 'xmin', 'ymin', 'xmax', 'ymax','pc_match','pc_match_flag']
new_annotation_df = pd.DataFrame(row_list, columns=column_name)


# define augmentor
augmentor = alb.Compose([alb.ChannelShuffle(p=1)],
                    bbox_params=alb.BboxParams(format='pascal_voc'))
    
    

    
# load image
img = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

# coleta o nome da imagem
image_name = createNameImage(image_path)

#filtra apenas as annotations referentes a imagem
image_annotation_df = ori_annotation_df[ori_annotation_df['filename']==image_name]

# cria bboxes
bboxes_class, bboxes = createBBoxesList(image_annotation_df)

print(bboxes_class)

imagem_orig = img

#for obj in bboxes_class:
    #cv2.rectangle(imagem_orig, (obj[0], obj[1]), (obj[2], obj[3]), (0, 255, 0), 10)

showImage(imagem_orig)

# realiza o augmentation
AugAnnotationDf = generateAugmentation(img, bboxes_class, augmentor, N, OUTPUT_PATH, image_name)



new_annotation_df = pd.concat([new_annotation_df, AugAnnotationDf])

print('Saving final_full_aug_annotation_df')

# Salva o csv com as Annotations de todas as imagens augmentadas
new_annotation_df = new_annotation_df.reset_index(drop=True)
new_annotation_df.to_csv(OUTPUT_ANNOTATION_PATH, encoding='utf-8', index=False)

In [ ]:
bboxes_class[0][0]

In [ ]:
IMAGE_PATH = "C:\\NEW_TCC\\dataset\\windows_image\\positive\\augmentation\\brancas\\cow664_w_28.JPG"

In [ ]:
image = cv2.imread(IMAGE_PATH)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

In [ ]:
transform = alb.Compose(
    [alb.HorizontalFlip(p=0.5)],
    bbox_params=alb.BboxParams(format='pascal_voc'),
)

In [ ]:
random.seed(7)
transformed = transform(image=image, bboxes=bboxes, category_ids=category_ids)
visualize(
    transformed['image'],
    transformed['bboxes'],
    transformed['category_ids'],
    category_id_to_name,
)